In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# ***Writing into a dataframe***

In [138]:
# This is an array to store data
data = []

for file_number in range(1, 16):
    file_name = f'match_{file_number}.json'

    with open(file_name, 'r') as file:
        match_data = json.load(file)

    # Loop through json files
    for match in match_data['data']['matches']:
        
        # Extract match ID
        match_id = match['attributes']['id']
        # Extract basic information about the match
        mode_name = match['metadata']['modeName']
        map_name = match['metadata']['mapName']
        season_name = match['metadata']['seasonName']
        agent_played = match['segments'][0]['metadata']['agentName']
        result = match['segments'][0]['metadata']['hasWon']
        timestamp = match['metadata']['timestamp']
        # Extract in-depth statistics about the match
        playtime = match['segments'][0]['stats']['playtime']['value']
        rounds_played = match['segments'][0]['stats']['roundsPlayed']['value']
        rounds_won = match['segments'][0]['stats']['roundsWon']['value']
        rounds_lost = match['segments'][0]['stats']['roundsLost']['value']
        kills = match['segments'][0]['stats']['kills']['value']
        deaths = match['segments'][0]['stats']['deaths']['value']
        assists = match['segments'][0]['stats']['assists']['value']
        total_damage_dealt = match['segments'][0]['stats']['damage']['value']
        total_damage_received = match['segments'][0]['stats']['damageReceived']['value']
        c_skill_casts = match['segments'][0]['stats']['grenadeCasts']['value']
        q_skill_casts = match['segments'][0]['stats']['ability1Casts']['value']
        e_skill_casts = match['segments'][0]['stats']['ability2Casts']['value']
        ultimate_casts = match['segments'][0]['stats']['ultimateCasts']['value']
        first_bloods = match['segments'][0]['stats']['firstBloods']['value']
        first_deaths = match['segments'][0]['stats']['firstDeaths']['value']
        last_deaths = match['segments'][0]['stats']['lastDeaths']['value']
        survived_rounds = match['segments'][0]['stats']['survived']['value']
        traded_rounds = match['segments'][0]['stats']['traded']['value']
        KAST = match['segments'][0]['stats']['kAST']['value'] # Kills/Assists/Survived/Traded ratio
        flawless_rounds = match['segments'][0]['stats']['flawless']['value']
        thrifty_rounds = match['segments'][0]['stats']['thrifty']['value']
        aces = match['segments'][0]['stats']['aces']['value']
        team_aces = match['segments'][0]['stats']['teamAces']['value']
        clutches_won = match['segments'][0]['stats']['clutches']['value']
        clutches_lost = match['segments'][0]['stats']['clutchesLost']['value']
        plants_down = match['segments'][0]['stats']['plants']['value'] # Number of times I was the one who plant the spike
        defuses = match['segments'][0]['stats']['defuses']['value'] # Number of times I was the one who defuses the spike
        headshot_percent = match['segments'][0]['stats']['headshotsPercentage']['value']
        damage_per_round = match['segments'][0]['stats']['damagePerRound']['value']
        kd_ratio = match['segments'][0]['stats']['kdRatio']['value']
        # Append all the extracted data to the list datum
        data.append([match_id, mode_name, map_name, season_name, agent_played, result, timestamp,
                    playtime, rounds_played, rounds_won, rounds_lost, kills, deaths, assists,
                    total_damage_dealt, total_damage_received, c_skill_casts, q_skill_casts,
                    e_skill_casts, ultimate_casts, first_bloods, first_deaths, last_deaths, 
                    survived_rounds, traded_rounds, KAST, flawless_rounds, thrifty_rounds,
                    aces, team_aces, clutches_won, clutches_lost, plants_down, defuses,
                    headshot_percent, damage_per_round, kd_ratio])
        
# This is my main dataframe for analysis
performance_df = pd.DataFrame(columns = ['ID', 'Mode', 'Map', 'Season', 'Agent', 'Win/Loss', 'Timestamp',
                                         'Playtime', 'Rounds', 'Rounds won', 'Rounds lost',
                                         'Kills', 'Deaths', 'Assists', 'Total damage', 'Total damage received', 
                                         'Grenades casts', 'Q Skill casts', 'E Skill casts', 'Ultimate casts',
                                         'First bloods', 'First deaths', 'Last deaths',
                                         'Survived rounds', 'Traded rounds', 'KAST', 'Flawless rounds',
                                         'Thrifties', 'Aces', 'Team aces', 'Clutches', 'Clutches lost',
                                         'Plants', 'Defuses', '%HS', 'Damage per round', 'K/D ratio'], 
                            data = data)
performance_df.head()

,ID,Mode,Map,Season,Agent,Win/Loss,Timestamp,Playtime,Rounds,Rounds won,Rounds lost,Kills,Deaths,Assists,Total damage,Total damage received,Grenades casts,Q Skill casts,E Skill casts,Ultimate casts,First bloods,First deaths,Last deaths,Survived rounds,Traded rounds,KAST,Flawless rounds,Thrifties,Aces,Team aces,Clutches,Clutches lost,Plants,Defuses,%HS,Damage per round,K/D ratio
0,6d5bf1ac-bfb3-4f90-a5a3-ee07f3b93973,Competitive,Fracture,V25: A1,Astra,False,2025-02-27T05:03:20.511+00:00,2196,21,8,13,9,17,7,2209,2642,1.0,5.0,49.0,3.0,1,1,8,6,0,67.0,1,0,0,0,NaN,6.0,1,1,15.909091,105.190476,0.529412
1,3eac0c76-c598-491b-9d44-da7560f1fddb,Competitive,Pearl,V25: A1,Astra,True,2025-02-27T04:27:16.287+00:00,1955,17,13,4,18,4,3,2581,813,3.0,7.0,48.0,2.0,0,0,2,13,0,100.0,1,0,0,0,3.0,2.0,0,0,25.531915,151.823529,4.500000
2,cb96a64e-bb42-4355-8dfa-e58e16af1291,Competitive,Fracture,V25: A1,Cypher,False,2025-02-27T03:42:08.207+00:00,2342,20,7,13,17,14,3,3145,2639,23.0,11.0,19.0,3.0,2,0,5,6,1,65.0,0,0,0,0,2.0,5.0,0,0,21.212121,157.250000,1.214286
3,c0d1e307-7a3d-4b0f-97f0-bebb947d9b12,Competitive,Lotus,V25: A1,Tejo,True,2025-02-26T16:56:44.392+00:00,1921,18,13,5,14,10,3,2174,2044,16.0,14.0,22.0,2.0,1,1,2,8,3,78.0,0,0,0,0,1.0,2.0,1,0,19.512195,120.777778,1.400000
4,4ad13992-22e0-4b14-8721-13832340dbf4,Competitive,Pearl,V25: A1,Viper,False,2025-02-25T15:51:15.42+00:00,1772,17,4,13,11,14,4,2011,2534,5.0,12.0,15.0,1.0,0,1,2,3,2,65.0,0,0,0,0,1.0,3.0,0,0,9.090909,118.294118,0.785714


In [140]:
performance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     270 non-null    object 
 1   Mode                   270 non-null    object 
 2   Map                    270 non-null    object 
 3   Season                 270 non-null    object 
 4   Agent                  270 non-null    object 
 5   Win/Loss               270 non-null    bool   
 6   Timestamp              270 non-null    object 
 7   Playtime               270 non-null    int64  
 8   Rounds                 270 non-null    int64  
 9   Rounds won             270 non-null    int64  
 10  Rounds lost            270 non-null    int64  
 11  Kills                  270 non-null    int64  
 12  Deaths                 270 non-null    int64  
 13  Assists                270 non-null    int64  
 14  Total damage           270 non-null    int64  
 15  Total 

In [141]:
performance_df.to_csv('my_personal_data_(1).csv')

# **Data visualization**

In [136]:
with open('match_1.json', 'r') as file:
    sample = json.load(file)

sample['data']['matches'][0]['metadata']['timestamp']

'2025-02-27T05:03:20.511+00:00'